## Persistent와 Non-Persistent HTTP, 그리고 Web Page 완성하기
### - Fetching the base HTML and included objects 

## Let's start with OOP
module ```mylib.py:```
- 먼저 Request class와 Connection class를 정의해 보자.
- 주어진 server와 TCP connection을 열고 Connection object를 return하는 함수를 생각해 보자.

In [9]:
from urllib.request import urlparse
import socket

class Request:
    def __init__(self, url, headers=None):
        """Create Request object from url"""
        r = urlparse(url)
        if r.scheme != 'http': raise NotImplementedError(r.scheme)
        self.server = r.hostname, r.port if r.port else 80
        self.path = r.path + '?' + r.query if r.query else r.path
        self.headers = {'Host': r.hostname}
        if headers:
            self.add_headers(headers)

    def add_headers(self, headers):
        """Add new headers of dict type
        """
        self.headers.update(headers)

    def build(self):
        """Build a request message
        """
        l = []
        l.append('GET {} HTTP/1.1'.format(self.path))
        for key, value in self.headers.items():
            l.append('{}: {}'.format(key, value))
        message = '\r\n'.join(l) + '\r\n\r\n'
        return message.encode('utf-8')

class Response:
    def __init__(self, status, headers, contents):
        self.status, self.headers, self.contents = status, headers, contents

class Connection:
    def __init__(self, server, sock, infile):
        """Session open by http_open function
        """
        self.server, self.sock, self.infile = server, sock, infile

    def close(self):
        if not self.sock._closed:
            self.infile.close()
            self.sock.close()

    def send(self, request):
        message = request.build()
        print('Sending request:', message, sep='\n')
        return self.sock.sendall(message)

    def get_headers(self):
        """Parse HTTP response message and get status and headers from it
        """

        def parse_headers(file):
            """extract headers as a dict
            """
            headers = {}
            for line in file:
                if line == b'\r\n':  # end of headers
                    break
                header = line.decode().strip()  # remove leading and trailing white spaces
                key, value = header.split(':', maxsplit=1)
                headers[key] = value.strip()
            return headers

        status_line = self.infile.readline()
        if status_line.startswith(b'HTTP/1.1'):
            status, status_phrase = status_line.decode().strip().split(maxsplit=2)[1:]
        else:
            raise SyntaxError('Not starting with status line')
        print('Status:', status, status_phrase)
        headers = parse_headers(self.infile)
        print('Headers:', headers)
        return status, headers

    def read(self):
        """Read contents according to header definitions"""

        def read_chunked(infile):
            # Chunked transfer encoding for streaming data
            # See https://en.wikipedia.org/wiki/Chunked_transfer_encoding
            chunks = []
            while True:
                hex_str = infile.readline().strip()
                chunk_len = int(hex_str, 16)
                print("chunk", chunk_len)
                if chunk_len == 0:
                    break
                chunk = infile.read(chunk_len)
                chunks.append(chunk)
                infile.readline()  # skip CRLF
            contents = b''.join(chunks)
            infile.readline()  # skip CRLF
            return contents

        status, headers = self.get_headers()
        content_len = headers.get('Content-Length')
        if content_len:
            # If Content-Length header exists, read the content-length bytes
            contents = self.infile.read(int(content_len))
        elif headers.get('Transfer-Encoding') == 'chunked':
            contents = read_chunked(self.infile)
        else:
            # otherwiese, read until server closing
            contents = self.infile.read()
        print('Contents:')
        print(contents[:40])
        print('...')
        print(contents[-40:])
        return Response(status, headers, contents)

def http_open(server):
    print('open new connection to server', server)
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect(server)
    infile = sock.makefile('rb')  # convert incoming socket to file-like object
    return Connection(server, sock, infile)

### Non-persistent HTTP
위 module을 활용해서 non-persistent HTTP mode로 web contents를 가져오자.

> Request object에서 header들은 ```dict``` type으로 표현된다.  ```add_headers``` method로 header들을 추가 할 수 있으며, ```build``` method로 request message를 생성할 수 있다.

> server로 하여금 response를 보낸 후 즉각 connection을 close할 것을 요청하기 위한 header를 추가했다. 

위 모듈을 Python IDE에서 사용하려면 import하라.
```Python
from mylib import Request, http_open
```

In [15]:
# url = "http://mclab.hufs.ac.kr/test/index.html"
url = "http://mclab.hufs.ac.kr/wiki/Lectures/CN/2018"
request = Request(url) # non_persistent
print(request.headers)
print(request.server)
print(request.build())

{'Host': 'mclab.hufs.ac.kr'}
('mclab.hufs.ac.kr', 80)
b'GET /wiki/Lectures/CN/2018 HTTP/1.1\r\nHost: mclab.hufs.ac.kr\r\n\r\n'


Web server와 TCP 연결해서 request message를 보내자. ```send``` method는 ```Request``` object에서 request message를 생성하여 socket으로 송신한다.

Response message는 open된 connection으로 먼저 status와 header들을 가져와야 한다.
- status: ```int```, headers: ```dict``` type

Header에는 Content-Length 등 실제 content를 분리하는데 필요한 정보가 들어 있다.
Non-persistent mode인 경우는 더 이상 connection을 유지할 필요가 없기 때문에 connection을 close해 줘야 한다.

In [16]:
conn = http_open(request.server)
conn.send(request)
response = conn.read()
conn.close()

open new connection to server ('mclab.hufs.ac.kr', 80)
Sending request:
b'GET /wiki/Lectures/CN/2018 HTTP/1.1\r\nHost: mclab.hufs.ac.kr\r\n\r\n'
Status: 200 OK
Headers: {'Date': 'Thu, 11 Oct 2018 01:23:55 GMT', 'Server': 'Apache/2.2.22 (Ubuntu)', 'X-Powered-By': 'PHP/5.3.10-1ubuntu3.10', 'X-Content-Type-Options': 'nosniff', 'Content-language': 'en', 'Vary': 'Accept-Encoding,Cookie', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Cache-Control': 'private, must-revalidate, max-age=0', 'Last-Modified': 'Thu, 11 Oct 2018 00:37:42 GMT', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html; charset=UTF-8'}
chunk 32011
chunk 0
Contents:
b'<!DOCTYPE html>\n<html lang="en" dir="ltr'
...
b'ved in 0.132 secs. -->\n\t</body>\n</html>\n'


In [17]:
print(response.status)
print(response.headers)
print(response.contents[:80])

200
{'Date': 'Thu, 11 Oct 2018 01:23:55 GMT', 'Server': 'Apache/2.2.22 (Ubuntu)', 'X-Powered-By': 'PHP/5.3.10-1ubuntu3.10', 'X-Content-Type-Options': 'nosniff', 'Content-language': 'en', 'Vary': 'Accept-Encoding,Cookie', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Cache-Control': 'private, must-revalidate, max-age=0', 'Last-Modified': 'Thu, 11 Oct 2018 00:37:42 GMT', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html; charset=UTF-8'}
b'<!DOCTYPE html>\n<html lang="en" dir="ltr" class="client-nojs">\n<head>\n<title>Lec'


### Persistent HTTP mode를 이용할 경우
HTTP/1.1에서는 default가 persistent mode이므로 추가 header 없이 `Request` object을 생성하면 충분하다.

다만, open된 connection은 다시 사용될 수 있으므로 close해서는 안된다.

In [20]:
base_url = "http://mclab.hufs.ac.kr/test/index.html"
request = Request(base_url)
conn = http_open(request.server)
conn.send(request)
base_html = conn.read()
print(conn.server)

open new connection to server ('mclab.hufs.ac.kr', 80)
Sending request:
b'GET /test/index.html HTTP/1.1\r\nHost: mclab.hufs.ac.kr\r\n\r\n'
Status: 200 OK
Headers: {'Date': 'Thu, 11 Oct 2018 01:35:43 GMT', 'Server': 'Apache/2.2.22 (Ubuntu)', 'Last-Modified': 'Tue, 19 Sep 2017 06:13:15 GMT', 'ETag': '"1e982f-569-55984c1337a5f"', 'Accept-Ranges': 'bytes', 'Content-Length': '1385', 'Vary': 'Accept-Encoding', 'Content-Type': 'text/html'}
Contents:
b'<html>\n<head>\n<title>Test Page</title>\n<'
...
b'/td>\n    </tr>\n</tbody></table>\n</html>\n'
('mclab.hufs.ac.kr', 80)


같은 server에 존재하는 object인 경우, 
위 connection을 사용하면 새로 connection 만드는데 걸리는 시간(RTT)을 절약할 수 있다.

In [21]:
obj_url = 'http://mclab.hufs.ac.kr/test/s3test2.gif'
# obj_url = 'http://ice.hufs.ac.kr/hufs-image01.jpg'
obj_req = Request(obj_url)
if conn.server == obj_req.server:
    obj_conn = conn  # reuse this connection
else:
    obj_conn = http_open(obj_req.server) # new connection    
obj_conn.send(obj_req)
image = obj_conn.read()

Sending request:
b'GET /test/s3test2.gif HTTP/1.1\r\nHost: mclab.hufs.ac.kr\r\n\r\n'
Status: 200 OK
Headers: {'Date': 'Thu, 11 Oct 2018 01:35:47 GMT', 'Server': 'Apache/2.2.22 (Ubuntu)', 'Last-Modified': 'Thu, 03 Oct 2013 05:45:45 GMT', 'ETag': '"1ea143-d272-4e7cfb2748beb"', 'Accept-Ranges': 'bytes', 'Content-Length': '53874', 'Content-Type': 'image/gif'}
Contents:
b'GIF89a\x90\x01,\x01\xf7\x00\x00$,I\x8f\x93\xa4,J\x9c\xc7\xc6\xccq\x93\xd4[p\x9cJWl\x99\xaf\xdbMq\xc4'
...
b'8\xe1\x0fv`q\x84\xdb_n\xe1\x11\xd6N\\\x86`[&C\xfa\x93[5\xad\x02]\xae\xd7e^\xe6z\x95\x80-\x08\x08\x00;'


In [22]:
base_html.contents

b'<html>\n<head>\n<title>Test Page</title>\n<meta http-equiv="content-type" content="text/html; charset=UTF-8">\n</head>\n\n<body>\n<h1>Information and Communications Engineering</h1>\n<p><img src="http://ice.hufs.ac.kr/hufs-image01.jpg" border="0"></p>\n<p>Welcome to Dept. of Information and Communications Engineering</p>\n<p></p>\n<p>\xed\x95\x9c\xea\xb5\xad\xec\x99\xb8\xea\xb5\xad\xec\x96\xb4\xeb\x8c\x80\xed\x95\x99\xea\xb5\x90 \xec\xa0\x95\xeb\xb3\xb4\xed\x86\xb5\xec\x8b\xa0\xea\xb3\xb5\xed\x95\x99\xea\xb3\xbc</p>\n\n<h2>Blue Sky</h2>\n<h3>SKY 2</h3>\n<p><img src="s3test2.gif" border="0"></p>\n\n<h3>SKY 3</h3>\n<p><img src="s3test3.jpg" border="0"></p>\n    <tr>\n        <td width="0"><font size="1">&nbsp;</font></td>\n        <td width="0"><font size="1">&nbsp;</font></td>\n            <p><span style="font-size: 22pt;"><font color="#17365d" face="Impact">SKY 4</font></span></p>\n            <p><font size="1"><img src="s3test4.jpg" border="0"></font></p>\n        <td width="0"><fon

### Base HTML Contents에서 Image object의 URL 추출하기
HTML contents를 scan하면서 `img` tag 내의 `src` attribute를 extract하면 된다. Regular expression을 사용하여 간단히 수행할 수 있다.

In [23]:
from urllib.request import urlopen
import re

pattern = re.compile(r'<img.*?src=\"(.*?)\".*?>')
urls = pattern.findall(base_html.contents.decode())
print(urls)

['http://ice.hufs.ac.kr/hufs-image01.jpg', 's3test2.gif', 's3test3.jpg', 's3test4.jpg', 's3test5.jpg']


### Converting relative URL to absolute URL
위의 URL들은 base URL에 relative하게 표시될 수도 있다. 이들을 absolute URL로 변환하면, `Request` object을 만들 수 있고, 같은 server에 존재하는지 여부도 확인할 수 있겠다.

In [24]:
from urllib.parse import urljoin

print(urljoin(base_url, 's3test2.gif'))
print(urljoin(base_url, 'http://ice.hufs.ac.kr/hufs-image01.jpg'))

http://mclab.hufs.ac.kr/test/s3test2.gif
http://ice.hufs.ac.kr/hufs-image01.jpg


In [25]:
absolute_urls = [urljoin(base_url, url) for url in urls]
print(absolute_urls)

['http://ice.hufs.ac.kr/hufs-image01.jpg', 'http://mclab.hufs.ac.kr/test/s3test2.gif', 'http://mclab.hufs.ac.kr/test/s3test3.jpg', 'http://mclab.hufs.ac.kr/test/s3test4.jpg', 'http://mclab.hufs.ac.kr/test/s3test5.jpg']


주의:
> 다른 일로 지체(예를 들어 0.5초 지연)하게 되면, server가 connection을 close할 수 있다.
> 따라서, 이 connection을 사용하려면, 같은 server에 존재하는 object들을 먼저 처리해야 안전하다.
> 예을 들어, mclab.hufs.ac.kr에 있는 URL들에 대해 먼저 request를 보내야 할 것이다.

### Pipelining in Persistent HTTP
Persistent HTTP mode에서 object들에 대한 request/response 순서는 다음과 같다.
> request-1 >> response-1 >> ... >> request-n >> response-n

만일, base html과 같은 server에 존재하는 object들에 대해서는 
다음과 같이 request들을 모두 보낸 후
> request-1 >> ... >> request-n

response messgage를 순서대로 받을 수 있다.
> response-1 >> ... >> response-n

Pipelining 방식으로 주고 받을 때, delay가 감소할까? 보통 naver와 같은 포털의 경우 한 개의 web page를 rendering하기 위해서 같은 server에서 수십개의 image와 javescript들을 fetch해야 한다.